# 作業 : (Kaggle)鐵達尼生存預測
https://www.kaggle.com/c/titanic

# [作業目標]
- 試著模仿範例寫法, 在鐵達尼生存預測中, 觀察計數編碼與特徵雜湊的效果

# [作業重點]
- 仿造範例, 完成計數編碼以及搭配邏輯斯迴歸的預測 (In[4], Out[4], In[5], Out[5]) 
- 仿造範例, 完成雜湊編碼, 以及計數編碼+雜湊編碼 搭配邏輯斯迴歸的預測 (In[6], Out[6], In[7], Out[7]) 
- 試著回答上述執行結果的觀察

# 作業1
* 參考範例，將鐵達尼的艙位代碼( 'Cabin' )欄位使用特徵雜湊 / 標籤編碼 / 計數編碼三種轉換後，  
與其他類別型欄位一起預估生存機率

In [138]:
# 做完特徵工程前的所有準備 (與前範例相同)
import pandas as pd
import numpy as np
import copy, time
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

data_path = 'data/'
df_train = pd.read_csv(data_path + 'titanic_train.csv')
df_test = pd.read_csv(data_path + 'titanic_test.csv')

train_Y = df_train['Survived']
ids = df_test['PassengerId']
df_train = df_train.drop(['PassengerId', 'Survived'] , axis=1)
df_test = df_test.drop(['PassengerId'] , axis=1)
df = pd.concat([df_train,df_test])
df.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [139]:
df.shape

(1309, 10)

In [140]:
#只取類別值 (object) 型欄位, 存於 object_features 中
object_features = []
for dtype, feature in zip(df.dtypes, df.columns):
    if dtype == 'object':
        object_features.append(feature)
print(f'{len(object_features)} Object Features : {object_features}\n')

# 只留類別型欄位
df = df[object_features]
df = df.fillna('None')
train_num = train_Y.shape[0]
df.head()

5 Object Features : ['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked']



,Name,Sex,Ticket,Cabin,Embarked
0,"Braund, Mr. Owen Harris",male,A/5 21171,None,S
1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,PC 17599,C85,C
2,"Heikkinen, Miss. Laina",female,STON/O2. 3101282,None,S
3,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,113803,C123,S
4,"Allen, Mr. William Henry",male,373450,None,S


# 作業2
* 承上題，三者比較效果何者最好?

In [141]:
# 對照組 : 標籤編碼 + 邏輯斯迴歸
df_temp = pd.DataFrame()
for c in df.columns:
    df_temp[c] = LabelEncoder().fit_transform(df[c])
train_X = df_temp[:train_num]
estimator = LogisticRegression()
print(cross_val_score(estimator, train_X, train_Y, cv=5).mean())
df_temp.head()

0.7800138095537004


,Name,Sex,Ticket,Cabin,Embarked
0,155,1,720,185,3
1,286,0,816,106,0
2,523,0,914,185,3
3,422,0,65,70,3
4,22,1,649,185,3


In [142]:
# 加上 'Cabin' 欄位的計數編碼
df_cabin = df.groupby(["Cabin"])["Name"].agg({"count"}).reset_index()
df_cabin =pd.merge(df, df_cabin, on="Cabin", how="outer")
df_cabin.sample(10)

,Name,Sex,Ticket,Cabin,Embarked,count
679,"Markun, Mr. Johann",male,349257,None,S,1014
330,"Zimmerman, Mr. Leo",male,315082,None,S,1014
1058,"Porter, Mr. Walter Chamberlain",male,110465,C110,S,1
809,"Peacock, Mrs. Benjamin (Edith Nile)",female,SOTON/O.Q. 3101315,None,S,1014
1123,"Penasco y Castellana, Mrs. Victor de Satode (M...",female,PC 17758,C65,C,2
768,"Buckley, Mr. Daniel",male,330920,None,Q,1014
1046,"Brown, Miss. Amelia ""Mildred""",female,248733,F33,S,4
811,"Touma, Master. Georges Youssef",male,2650,None,C,1014
181,"Mellors, Mr. William John",male,SW/PP 751,None,S,1014
1229,"Calderhead, Mr. Edward Pennington",male,PC 17476,E24,S,2


In [143]:
# 'Cabin'計數編碼 + 邏輯斯迴歸
for i in df.columns:
    df_cabin[i] = LabelEncoder().fit_transform(df_cabin[i])
estimator = LogisticRegression()
train_X = df_cabin.iloc[:train_num, :]
print(cross_val_score(estimator, train_X, train_Y, cv=5).mean())
print(df_cabin.sample(5))

0.6150461364634989
      Name  Sex  Ticket  Cabin  Embarked  count
1178  1142    0     627    123         0      2
311    616    0     675    185         3   1014
499     78    0     260    185         0   1014
277   1301    1     866    185         0   1014
785    957    1     404    185         3   1014


In [144]:
# 'Cabin'特徵雜湊 + 邏輯斯迴歸
for c in object_features:
    df[c] = LabelEncoder().fit_transform(df[c])
# 這邊的雜湊編碼, 是直接將 'Ticket' 的名稱放入雜湊函數的輸出數值, 為了要確定是緊密(dense)特徵, 因此除以10後看餘數
# 這邊的 10 是隨機選擇, 不一定要用 10, 同學可以自由選擇購小的數字試看看. 基本上效果都不會太好
df['Cabin_Hash'] = df['Cabin'].map(lambda x:hash(x) % 10)
train_X = df[:train_num]
estimator = LogisticRegression()
print(cross_val_score(estimator, train_X, train_Y, cv=5).mean())
df.head()

0.7833657648609629


/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-reg

,Name,Sex,Ticket,Cabin,Embarked,Cabin_Hash
0,155,1,720,185,3,5
1,286,0,816,106,0,6
2,523,0,914,185,3,5
3,422,0,65,70,3,0
4,22,1,649,185,3,5


In [145]:
# 'Cabin'計數編碼 + 'Cabin'特徵雜湊 + 邏輯斯迴歸
df_cabin["hash"] = df["Cabin_Hash"].values
estimator = LogisticRegression()
train_X = df_cabin.iloc[:train_num, :]
print(cross_val_score(estimator, train_X, train_Y, cv=5).mean())
print(df_cabin.sample(5))

0.6610696127047895
      Name  Sex  Ticket  Cabin  Embarked  count  hash
99     846    1     174    185         0   1014     5
438   1147    1     488    185         3   1014     0
90      95    1     370    185         3   1014     5
393    893    1     743    185         3   1014     2
1220   737    0     187     43         3      2     5


In [146]:
# 'Cabin'計數編碼 + 邏輯斯迴歸, 0.6150461364634989
# 'Cabin'特徵雜湊 + 邏輯斯迴歸, 0.7833657648609629
# Cabin'計數編碼 + 'Cabin'特徵雜湊 + 邏輯斯迴歸, 0.6610696127047895